In [1]:
pip install linearmodels

     |████████████████████████████████| 1.2MB 4.7MB/s 
  Created wheel for linearmodels: filename=linearmodels-4.13-py2.py3-none-any.whl size=802535 sha256=3ff8eb629bb967f16297ebb78e7c8cc921496e56456094033c473d5c86cc9a12
  Stored in directory: /root/.cache/pip/wheels/92/33/3a/5efd7cb50597ff3dc29256ba54e36e2a2c820fd1d1069ea967
Successfully built linearmodels


In [0]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

In [3]:
file="https://github.com/VitorKamada/ECO7110/raw/master/Data/mus05surdata.dta"
df = pd.read_stata(file)
df.head()

,ssiratio,year03,age,educyr,drugexp,drugslf,totslf,totexp,medicare,medicaid,private,female,actlim,income,totchr,age2,noopexp,oopexp,totothr,loopexp,lnoopexp,ldrugexp,ltotothr
0,0.394506,1,85,15,1754,1754,1754,2054,1,0,1,1,0,27.882999,3,7225.0,300.0,1754.0,300.0,7.469654,5.703783,7.469654,5.703783
1,1.000000,1,83,8,240,240,381,1880,1,0,0,1,1,11.628000,2,6889.0,1499.0,381.0,1640.0,5.942800,7.312553,5.480639,7.402452
2,1.000000,1,82,11,453,102,102,656,1,1,0,1,0,3.456000,2,6724.0,554.0,102.0,203.0,4.624973,6.317165,6.115892,5.313206
3,0.205162,1,77,13,1729,722,744,8350,1,0,1,0,0,38.973999,3,5929.0,7606.0,744.0,6621.0,6.612041,8.936692,7.455298,8.798001
4,0.216923,1,76,14,1814,510,817,3380,1,0,1,1,0,36.861000,1,5776.0,2563.0,817.0,1566.0,6.705639,7.848934,7.503290,7.356280


In [4]:
X1 = sm.add_constant(df[['age', 'age2', 
 'actlim', 'totchr', 'medicaid', 'private']])
X2 = sm.add_constant(df[['age', 'age2', 
 'actlim','totchr', 'educyr', 'private']])

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [5]:
OLS1 = sm.OLS(df.ldrugexp, X1,
 missing='drop').fit(cov_type='HC1')
print(OLS1.summary())

                            OLS Regression Results                            
Dep. Variable:               ldrugexp   R-squared:                       0.227
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     170.4
Date:                Thu, 08 Aug 2019   Prob (F-statistic):          4.52e-189
Time:                        21:51:06   Log-Likelihood:                -5100.5
No. Observations:                3285   AIC:                         1.021e+04
Df Residuals:                    3278   BIC:                         1.026e+04
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.4022      2.972     -1.481      0.1

In [6]:
OLS2 = sm.OLS(df.ltotothr, X2,
 missing='drop').fit(cov_type='HC1')
print(OLS2.summary())

                            OLS Regression Results                            
Dep. Variable:               ltotothr   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     103.2
Date:                Thu, 08 Aug 2019   Prob (F-statistic):          1.54e-119
Time:                        21:51:06   Log-Likelihood:                -6074.4
No. Observations:                3350   AIC:                         1.216e+04
Df Residuals:                    3343   BIC:                         1.221e+04
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.1414      3.853     -1.594      0.1

In [7]:
from linearmodels.system import SUR
from collections import OrderedDict
Equation = OrderedDict()
Equation['ldrugexp'] = {'dependent': df.ldrugexp, 'exog': X1}
Equation['ltotothr'] = {'dependent': df.ltotothr, 'exog': X2}
SUR_Reg = SUR(Equation).fit()
print(SUR_Reg)

                        System GLS Estimation Summary                         
Estimator:                        GLS   Overall R-squared:              0.1800
No. Equations.:                     2   Cov. Estimator:                 robust
No. Observations:                3251   Num. Constraints:                 None
Date:                Thu, Aug 08 2019                                         
Time:                        21:51:06                                         
                                                                              
                                                                              
               Equation: ldrugexp, Dependent Variable: ldrugexp               
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -3.8913     2.6290    -1.4802     0.1388     -9.0439      1.2614
age            0.2630     0.0702     3.7478     0.00

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [8]:
res1 = SUR_Reg.equations['ldrugexp'].resids
res2 = SUR_Reg.equations['ltotothr'].resids
np.corrcoef(res1, res2)

array([[1.        , 0.17427516],
       [0.17427516, 1.        ]])